In [96]:
import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'DB_and_Azure'))
import sql_db_functions as SQLf

### get data from server

In [97]:
conn, cursor = SQLf.sql_db_functions.connect_sql()

In [98]:
query = "SELECT product_characteristics.id, Brand_id , Detail, Summary, Brand FROM product_characteristics INNER JOIN Products ON product_characteristics.Brand_id = Products.Brand_Prod_id ;"
cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()
rows[0]

(2,
 2,
 '*Type: Shirt  \n*Fit: Regular fit  \n*Sleeve style: Long sleeve  \n*Neckline: Not Applicable  \n*Material: Cotton  \n*Formality: Business Casual  \n*Season: Spring, Summer, Autumn  \n*Colors: Light blue (100%)  \n*Texture: Smooth  \n*Transparency: Opaque  \n*Details and Embellishments: Buttons, GG Shadow print  \n*Shape: Fitted  \n*Length: Hip-length  \n*Collar Style: Button-down  \n*Sleeve Style: Long  \n*Patterns: Geometric (GG Shadow)  \n*Patterns placement: All over  \n*Fluidity of fabric: Medium  \n*Fabric weight: Light  \n*Pocket Presence: No  \n*Pocket placement: Not applicable  \n*Pocket size: Not applicable  \n*Breathability: High  \n*Occasion Suitability: Business casual  \n*Lapel: Not applicable',
 'The shirt depicted in the images is a Gucci Oxford cotton shirt featuring the GG Shadow monogram pattern. The fit appears to be regular, offering a comfortable yet slightly tailored silhouette. It has long sleeves with buttoned cuffs and a classic button-down closure. T

In [99]:
conn, cursor = SQLf.sql_db_functions.connect_sql()

In [100]:
query = "SELECT Brand_id, base64 FROM product_img ;"
cursor.execute(query)

# Fetch the rows
rows_img = cursor.fetchall()
#rows_img

In [101]:
len(rows_img)

629

In [102]:
SQLf.sql_db_functions.close_connection_db(conn=conn,cursor=cursor)

### organize data

In [103]:
import pandas as pd

In [104]:
df = pd.DataFrame(rows ,columns= ['Id','prod_id','descripcion', 'Summary','Brand'])

In [105]:
df.head()

,Id,prod_id,descripcion,Summary,Brand
0,2,2,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,The shirt depicted in the images is a Gucci Ox...,Gucci
1,4,4,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,The showcased shirt is a luxurious Gucci creat...,Gucci
2,6,6,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,The shirt showcased in the images is a Gucci p...,Gucci
3,8,8,*Type: Top\n*Fit: Regular fit\n*Sleeve style: ...,The showcased top from Gucci is a floral lace ...,Gucci
4,11,10,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,"The Gucci cotton poplin shirt, inspired by sum...",Gucci


In [106]:
df['descripcion'] = df['descripcion'].str.replace('\n*', ' / ')
df['descripcion'] = df['descripcion'].str.replace(r'\s+', ' ')
df['descripcion'] = df['descripcion'].str.replace('*', ' ')

df.head(5)

,Id,prod_id,descripcion,Summary,Brand
0,2,2,Type: Shirt / Fit: Regular fit / Sleeve s...,The shirt depicted in the images is a Gucci Ox...,Gucci
1,4,4,Type: Shirt / Fit: Regular fit / Sleeve s...,The showcased shirt is a luxurious Gucci creat...,Gucci
2,6,6,Type: Shirt / Fit: Regular fit / Sleeve s...,The shirt showcased in the images is a Gucci p...,Gucci
3,8,8,Type: Top / Fit: Regular fit / Sleeve style: ...,The showcased top from Gucci is a floral lace ...,Gucci
4,11,10,Type: Shirt / Fit: Regular fit / Sleeve s...,"The Gucci cotton poplin shirt, inspired by sum...",Gucci


In [107]:
df['Complete_description'] = df['Summary'] + ' /' + "Brand: " + df['Brand'] + " /" + df['descripcion']
df.head()

,Id,prod_id,descripcion,Summary,Brand,Complete_description
0,2,2,Type: Shirt / Fit: Regular fit / Sleeve s...,The shirt depicted in the images is a Gucci Ox...,Gucci,The shirt depicted in the images is a Gucci Ox...
1,4,4,Type: Shirt / Fit: Regular fit / Sleeve s...,The showcased shirt is a luxurious Gucci creat...,Gucci,The showcased shirt is a luxurious Gucci creat...
2,6,6,Type: Shirt / Fit: Regular fit / Sleeve s...,The shirt showcased in the images is a Gucci p...,Gucci,The shirt showcased in the images is a Gucci p...
3,8,8,Type: Top / Fit: Regular fit / Sleeve style: ...,The showcased top from Gucci is a floral lace ...,Gucci,The showcased top from Gucci is a floral lace ...
4,11,10,Type: Shirt / Fit: Regular fit / Sleeve s...,"The Gucci cotton poplin shirt, inspired by sum...",Gucci,"The Gucci cotton poplin shirt, inspired by sum..."


In [108]:
documents = df['Complete_description'].tolist()
id = df['Id'].astype('str').tolist()
metadata = [{"source": id} for id in df['Id'].astype('str').tolist()]

In [109]:
df_image = pd.DataFrame(rows_img, columns=['prod_id', 'base64'])

In [110]:
df_image.head(2)

,prod_id,base64
0,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
1,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."


In [111]:
df_image.shape

(629, 2)

In [112]:
del rows, rows_img

### Get images 

In [113]:
import base64
from io import BytesIO
from PIL import Image

In [114]:
def base64_to_image(base64_string):
    # Remove the data URI prefix if present
    if "data:image" in base64_string:
        base64_string = base64_string.split(",")[1]

    # Decode the Base64 string into bytes
    image_bytes = base64.b64decode(base64_string)
    return image_bytes

In [115]:
def create_image_from_bytes(image_bytes):
    # Create a BytesIO object to handle the image data
    image_stream = BytesIO(image_bytes)

    # Open the image using Pillow (PIL)
    image = Image.open(image_stream)
    return image

In [116]:
def resize_image(image, new_width):
    # Open the image

    # Calculate the new height while maintaining the aspect ratio
    original_width, original_height = image.size
    aspect_ratio = original_height / original_width
    new_height = int(new_width * aspect_ratio)

    # Resize the image
    resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)

    return resized_image


In [117]:
df_image.head()

,prod_id,base64
0,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
1,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
2,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
3,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
4,2,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."


In [118]:
df_image.iloc[42]

prod_id                                                   11
base64     data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ...
Name: 42, dtype: object

### Testing embeddings

In [119]:
import pandas as pd

In [120]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [121]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [122]:
embedding_function = SentenceTransformerEmbeddings(model_name='all-mpnet-base-v2')

### organized text 

In [123]:
df = pd.merge(
            df,
            df_image.groupby('prod_id')['base64'].apply(list).reset_index(),
            on='prod_id'
        )

In [124]:
df.prod_id = df.prod_id.astype(int)
df.sort_values('prod_id',ascending=True,inplace=True)
df.reset_index(drop=True, inplace= True)
df.prod_id = df.prod_id.astype('string')
df.head(5)

,Id,prod_id,descripcion,Summary,Brand,Complete_description,base64
0,1,1,Type: Shirt / Fit: Regular fit / Sleeve s...,The Gucci GG Shadow Silk Shirt is a luxurious ...,Gucci,The Gucci GG Shadow Silk Shirt is a luxurious ...,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
1,2,2,Type: Shirt / Fit: Regular fit / Sleeve s...,The shirt depicted in the images is a Gucci Ox...,Gucci,The shirt depicted in the images is a Gucci Ox...,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
2,3,3,Type: Top / Fit: Regular fit / Sleeve sty...,The top showcased in the images is a cropped s...,Gucci,The top showcased in the images is a cropped s...,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
3,4,4,Type: Shirt / Fit: Regular fit / Sleeve s...,The showcased shirt is a luxurious Gucci creat...,Gucci,The showcased shirt is a luxurious Gucci creat...,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
4,5,5,Type: Shirt / Fit: Regular fit / Sleeve s...,The shirt showcased in the images is a Gucci c...,Gucci,The shirt showcased in the images is a Gucci c...,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."


### chunks

In [125]:
df = df[['prod_id', 'Summary','descripcion','Brand','base64']]#.append(df[['prod_id','Summary', 'descripcion','Brand','base64']])

In [126]:
df.head()

,prod_id,Summary,descripcion,Brand,base64
0,1,The Gucci GG Shadow Silk Shirt is a luxurious ...,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
1,2,The shirt depicted in the images is a Gucci Ox...,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
2,3,The top showcased in the images is a cropped s...,Type: Top / Fit: Regular fit / Sleeve sty...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
3,4,The showcased shirt is a luxurious Gucci creat...,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
4,5,The shirt showcased in the images is a Gucci c...,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."


In [127]:
df.Brand.unique()

array(['Gucci', 'HM', 'Prada', 'Mango'], dtype=object)

In [128]:
df['Type'] = df['Brand'].apply(lambda x: 'Luxury' if (x == 'Gucci') or (x == 'Prada') else 'Retail')

In [129]:
df.head()

,prod_id,Summary,descripcion,Brand,base64,Type
0,1,The Gucci GG Shadow Silk Shirt is a luxurious ...,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
1,2,The shirt depicted in the images is a Gucci Ox...,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
2,3,The top showcased in the images is a cropped s...,Type: Top / Fit: Regular fit / Sleeve sty...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
3,4,The showcased shirt is a luxurious Gucci creat...,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
4,5,The shirt showcased in the images is a Gucci c...,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury


In [130]:
persist_directory = '../../../Chromadb/chroma-retail'

In [131]:
store = InMemoryStore()
id_key = 'doc_id'

In [132]:
min(df['base64'].apply(lambda x: len(x)))

1

In [133]:
df[ df['base64'].apply(lambda x: len(x)) == 1 ]

,prod_id,Summary,descripcion,Brand,base64,Type
130,131,The top showcased in the image is a fluid fabr...,Type: Top - Shirt / Fit: Regular fit / Sleeve...,Mango,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Retail


In [134]:
doc_ids = df.prod_id


descripcions = [
    Document(
        page_content= df.iloc[i].descripcion, 
        metadata={
            id_key:df.iloc[i].prod_id,
            'Type': df.iloc[i].Type
            }
    )
    for i in df.index
]


In [135]:
df.head()

,prod_id,Summary,descripcion,Brand,base64,Type
0,1,The Gucci GG Shadow Silk Shirt is a luxurious ...,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
1,2,The shirt depicted in the images is a Gucci Ox...,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
2,3,The top showcased in the images is a cropped s...,Type: Top / Fit: Regular fit / Sleeve sty...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
3,4,The showcased shirt is a luxurious Gucci creat...,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
4,5,The shirt showcased in the images is a Gucci c...,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury


In [136]:
final_df = pd.DataFrame()

for index, parent_row in df.iterrows(): 
    x = []
    for row in parent_row.descripcion.split('/'):
        x.append(row.split(':'))

    temp_dataframe = pd.DataFrame(x)
    temp_dataframe[0] = temp_dataframe[0].str.lower()
    temp_dataframe = temp_dataframe.sort_values(by=0)
    temp_dataframe = temp_dataframe.groupby(0)[1].apply(lambda x: ', '.join(x)).reset_index()
    temp_dataframe['prod_id'] = parent_row.prod_id
    temp_dataframe['type'] = parent_row.Type
    
        
    #temp_dataframe = temp_dataframe.set_index(0).T
    #temp_dataframe.columns = temp_dataframe.columns.str.lstrip()
    
    final_df = pd.concat([final_df,temp_dataframe], axis=0)


In [137]:
final_df

,0,1,prod_id,type
0,breathability,Medium,1,Luxury
1,collar style,Button-down,1,Luxury
2,colors,100% Dark Gray,1,Luxury
3,details and embellishments,"Buttons, GG Shadow print",1,Luxury
4,fabric weight,Light,1,Luxury
...,...,...,...,...
19,shape,Boxy,154,Retail
20,sleeve style,"Long , Long sleeve",154,Retail
21,texture,Smooth,154,Retail
22,transparency,Semi-transparent,154,Retail


In [139]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20,)

In [141]:
doc_ids = final_df.prod_id


descripcions = [
    Document(
        page_content= final_df.iloc[i][1], 
        metadata={
            id_key:final_df.iloc[i].prod_id,
            'Type': final_df.iloc[i].type,
            'category': final_df.iloc[i][0]         
            }
    )
    for i in final_df.index
]


In [142]:
doc_ids = df.prod_id


texts = [
    Document(
        page_content= df.iloc[i].Summary, 
        metadata={
            id_key:df.iloc[i].prod_id,
            'Type': df.iloc[i].Type
            }
    )
    for i in df.index
]


In [145]:
texts[1].page_content

'The shirt depicted in the images is a Gucci Oxford cotton shirt featuring the GG Shadow monogram pattern. The fit appears to be regular, offering a comfortable yet slightly tailored silhouette. It has long sleeves with buttoned cuffs and a classic button-down closure. The neckline is a standard collar style, adding to its formal appearance. The material is high-quality Oxford cotton, suitable for year-round wear, making it versatile for different seasons. The shirt is primarily in a light blue color, with the GG monogram pattern subtly interwoven in a slightly darker shade of blue, creating a sophisticated and understated texture. The fabric is not transparent and has a smooth texture, indicative of medium-weight cotton, ensuring good breathability. There are no additional embellishments or pockets, maintaining a clean and elegant look. The pattern placement is all-over, giving a uniform and balanced aesthetic. This shirt is suitable for both formal and semi-formal occasions, thanks t

In [143]:
text_child = []

for i in df.index:

    child_docs=child_splitter.create_documents(
            [texts[i].page_content],
            metadatas= 
                [{
                    'doc_id':texts[i].metadata['doc_id'],
                    'Type': df.iloc[i].Type
                }]
        )
    
    for _doc in child_docs:
        text_child.append(_doc)
    

In [146]:
vectorstore = Chroma.from_documents(
                documents=text_child,
                #ids=doc_ids,
                embedding=embedding_function,
                collection_name='RAG-Child',
                #metadatas=metadata,
                persist_directory=persist_directory
            )

In [147]:
vectorstore.similarity_search_with_score('Luxury')

[(Document(page_content='elegant and formal settings, offering a blend of luxury and understated sophistication.', metadata={'Type': 'Luxury', 'doc_id': '17'}),
  0.9962674379348755),
 (Document(page_content='a stylish day out, emphasizing both comfort and elegance in its design.', metadata={'Type': 'Luxury', 'doc_id': '5'}),
  1.0528672933578491),
 (Document(page_content='occasions due to its elegant design and material.', metadata={'Type': 'Retail', 'doc_id': '134'}),
  1.2317487322093055),
 (Document(page_content='social gatherings, embodying a perfect blend of classic design and high-end fashion.', metadata={'Type': 'Luxury', 'doc_id': '23'}),
  1.2378370761871338)]

In [148]:
vectorstore = Chroma.from_documents(
                documents=descripcions,
                #ids=doc_ids,
                embedding=embedding_function,
                collection_name='RAG-Child-Details',
                #metadatas=metadata,
                persist_directory=persist_directory
            )

### Already loaded !!!

In [ ]:
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
    search_kwargs={"k":5}
)

In [ ]:
docs = retriever.invoke(
    "HM"
)

In [ ]:
docs

In [ ]:
retriever.vectorstore.add_documents(texts)
#retriever.docstore.mset(list(zip(doc_ids, texts)))

In [ ]:
docs = retriever.invoke(
    "HM"
)

In [ ]:
docs

In [ ]:
df.iloc[1].Complete_description

### Talk to Chroma

In [ ]:
from apikey import apikey 
from langchain.chat_models import ChatOpenAI

In [ ]:
os.environ['OPENAI_API_KEY'] = apikey

In [ ]:
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

### lets set up the system

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
system_prompt = (
    "You are a Fashion specialist engine to sugest clothing pieces from context. "
    "If you don't know the answer to the question, say that you dont know "
    "answer concise."
    #"Explain why the selected options of the context are the best ones"
    "\n\n"
    "{context}"
)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | turbo_llm
    | StrOutputParser()
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(turbo_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
query = "Im looking for clothing peaces from Mango that could be similar to Gucci clothing"

In [ ]:
response = rag_chain.invoke({"input": query})
print(response["answer"])

In [ ]:
for document in response["context"]:
    print(document)
    print()

In [ ]:
df_image[df_image.prod_id == 41]

In [ ]:
image_bytes = base64_to_image(df_image.iloc[113].base64)
img = create_image_from_bytes(image_bytes=image_bytes)
img.resize((400,500))